In [ ]:
import numpy as np
import os
import pandas as pd
import torch
from PIL import Image, ImageFile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch.utils.data import Dataset, DataLoader
from intersection_over_union import intersection_over_union_wh
from non_max_suppression import nms

In [ ]:
#no errors while reading images
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [ ]:
#S is grid sizes
#C number of classes
class YOLODataset(Dataset):
    def __init__(
        self,
        csv_file,
        img_dir,
        label_dir,
        anchors,
        image_size=416,
        S=[13, 26, 52],
        C=20,
        transform = None
    ):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.image_size = image_size
        self.S = S
        self.anchors = torch.tensor(anchors[0] + anchors[1] + anchors[2])  # for all 3 scales
        self.num_anchors = self.anchors.shape[0]
        self.num_anchors_per_scale = self.num_anchors // 3
        self.C = C
        self.ignore_iou_thresh = 0.5
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        label_path = os.path.join(self.label_dir, self.annotations.iloc[index, 1])
        #bboxes = np.loadtxt(fname=label_path, delimiter=" ", ndmin=2).tolist() # [c,x,y,w,h]
        #this one is for the agumentation using Albumentations lib
        bboxes = np.roll(np.loadtxt(fname=label_path, delimiter=" ", ndmin=2), 4, axis=1).tolist() # [x,y,w,h,c]
        img_path = os.path.join(self.img_dir, self.annotations.iloc[index, 0])
        image = np.array(Image.open(img_path).convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=image, bboxes=bboxes)
            image = augmentations["image"]
            bboxes = augmentations["bboxes"]


        # Below assumes 3 scale predictions (as paper) and same num of anchors per scale
        targets = [torch.zeros((self.num_anchors // 3, S, S, 6)) for S in self.S]
        #[3,13,13,[p_o , x,y ,w,h,c]]
        # to specify which anchor box has the resbonsability for prediction in which scale
        # we determain that by the anchor who has the highest IOU with the original box
        for box in bboxes:
            #print (self.anchors)
            #print (torch.tensor(box[2:4]))
            #iou_anchors = intersection_over_union(torch.tensor(box[2:4]), self.anchors)
            iou_anchors = intersection_over_union_wh(torch.tensor(box[2:4]), self.anchors)
            #iou_anchors = ops.box_iou(self.anchors,torch.tensor(box[2:4]))
            #print (iou_anchors)
            anchor_indices = iou_anchors.argsort(descending=True, dim=0)
            x, y, width, height, class_label = box
            has_anchor = [False] * 3  # each scale should have one anchor
            
            for anchor_idx in anchor_indices:
                #check which anchor belong to which scale
                #scale_idx = anchor_idx // self.num_anchors_per_scale
                scale_idx = torch.div(anchor_idx, self.num_anchors_per_scale, rounding_mode='floor')
                
                anchor_on_scale = anchor_idx % self.num_anchors_per_scale
                #how many cells in the particualr scale becuase the input in the labels is in the whole image but we want it to specific to the cell
                #print (scale_idx)
                S = self.S[scale_idx]
                #which x and y cell
                i, j = int(S * y), int(S * x) # x = 0.5 , S = 13 -> int(6.5) = 6
                anchor_taken = targets[scale_idx][anchor_on_scale, i, j, 0]
                
                #if the anchor is not taken in this specific scale and specific bounding box
                if not anchor_taken and not has_anchor[scale_idx]:
                    targets[scale_idx][anchor_on_scale, i, j, 0] = 1
                    # the cordination within the cell between [0,1]
                    x_cell, y_cell = S * x - j, S * y - i  # s =13 ,x = 0.5-> 6.5 -6 = 0.5
                    width_cell, height_cell = (
                        width * S, #s=13 width = 0.5 , 6.5
                        height * S,
                    )  # can be greater than 1 since it's relative to cell
                    box_coordinates = torch.tensor(
                        [x_cell, y_cell, width_cell, height_cell]
                    )
                    targets[scale_idx][anchor_on_scale, i, j, 1:5] = box_coordinates
                    targets[scale_idx][anchor_on_scale, i, j, 5] = int(class_label)
                    has_anchor[scale_idx] = True

                # ignore other bounding boxes if they have an IOU > thresh but still there is a bounding box with a higher IOU for this scall
                elif not anchor_taken and iou_anchors[anchor_idx] > self.ignore_iou_thresh:
                    targets[scale_idx][anchor_on_scale, i, j, 0] = -1  # ignore prediction

        return image, tuple(targets)
